In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from tqdm.notebook import tqdm

In [4]:
import datetime
from datetime import timedelta

In [5]:
data =pd.read_json('data_v1_COM.json',orient='table')

data["방송날짜"] = pd.to_datetime(data["방송날짜"])

In [20]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://datalab.naver.com/keyword/trendSearch.naver")

df = data[data["상품군"]=="의류"]
uniquebrand = pd.unique(df["브랜드"])
Max_num = data[data["상품군"]=="의류"]["판매일수"].iloc[0]

for brand in uniquebrand:
    df2 = df[df["브랜드"]==brand].reset_index(drop=True)
    compare = df2.loc[0,"방송날짜"] ### 15일 간격을 계속 갱신해주기 위한 변수 , 이 FOR문에 있어야함
    for i in tqdm(range(0,len(df2))):

        if i != 0:  ### 첫번째 행이 아니면
            if (df2.loc[i,"방송날짜"] - compare).days<=15:
                continue
            else:
                compare = df2.loc[i,"방송날짜"]

        #if data.iloc[i,:]["상품군"]=="의류":
        end = df2.iloc[i,:]["방송날짜"]
        start = end - timedelta(days=np.float64(Max_num))
        ## 연도
        if start.year == 2019:
            start_year = "4"
        else:
            start_year = "3"
        
        if end.year==2020:
            end_year ="5"
        elif end.year == 2019:
            end_year = "4"
        else:
            end_year = "3"
        ## 월,일
        start_month ,start_day, end_month, end_day = str(start.month), str(start.day), str(end.month), str(end.day)
        if brand =="Nobrand":
            topic = df2.loc[i,"브랜드상품"]
        else:
            topic =df2.loc[i,"브랜드"]

        # 주제어 넣기
        elem = driver.find_element_by_id("item_keyword1")
        elem.send_keys(topic)
        
        # 세부 주제어 넣기
        elem = driver.find_element_by_id("item_sub_keyword1_1")
        elem.send_keys(topic)
        for j in range(0,len(df2.loc[i,"브랜드상품"])):
            elem.send_keys(",")
            elem.send_keys(df2.loc[i,"브랜드상품"][j])

        time.sleep(1)
        # 시작날짜 설정하기
        elem = driver.find_element_by_xpath("//*[@id='startYear']")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='startYearDiv']/ul/li["+start_year+"]/a")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='startMonth']")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='startMonthDiv']/ul/li["+start_month+"]/a")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='startDay']")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='startDayDiv']/ul/li["+start_day+"]/a")
        elem.click()
        # 종료날짜 설정하기
        time.sleep(1)
        elem = driver.find_element_by_xpath("//*[@id='endYear']")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='endYearDiv']/ul/li["+end_year+"]/a")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='endMonth']")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='endMonthDiv']/ul/li["+end_month+"]/a")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='endDay']")
        elem.click()
        elem = driver.find_element_by_xpath("//*[@id='endDayDiv']/ul/li["+end_day+"]/a")
        elem.click()
        time.sleep(1.5)

        # 검색버튼 클릭
        elem = driver.find_element_by_xpath("//*[@id='content']/div/div[2]/div[1]/div/form/fieldset/a/span")
        elem.click()
        time.sleep(1)

        # 파일 다운로드
        elem = driver.find_element_by_xpath("//*[@id='content']/div/div[1]/div[1]/div/div/div/div/div/div[1]/div[4]/a")
        elem.click()
        time.sleep(1.5)

        # 뒤로 가기 
        driver.back()
        time.sleep(2)

        #텍스트 지우기
        elem = driver.find_element_by_id("item_keyword1")
        elem.clear()
        elem = driver.find_element_by_id("item_sub_keyword1_1")
        elem.clear()
        

        print("%s %s %d %d"%(df2.loc[i,"브랜드"],df2.loc[i,"브랜드상품"],df2.loc[i,"방송_월"],df2.loc[i,"방송_일"]))

KeyboardInterrupt: 